<a href="https://colab.research.google.com/github/WISEPLAT/_NN/blob/main/3_%D1%82%D0%BE%D0%BD%D0%BA%D0%B0%D1%8F_%D0%BD%D0%B0%D1%81%D1%82%D1%80%D0%BE%D0%B9%D0%BA%D0%B0_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%BE%D0%B9_%D1%81%D0%B5%D1%82%D0%B8_%D0%B0%D0%BD%D1%81%D0%B0%D0%BC%D0%B1%D0%BB%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Dense, Flatten, Convolution2D, MaxPooling2D, Dropout, Concatenate
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values

In [ ]:
from keras.regularizers import l2 # L2-regularisation
l2_lambda = 0.0001
# W_regularizer == kernel_regularizer
# border_mode == padding
# init == kernel_initializer

In [ ]:
#https://stackoverflow.com/questions/51075666/how-to-implement-merge-from-keras-layers
#from keras.layers import add # for merging predictions in an ensemble
# merge == add
ens_models = 2 # we will train three separate models on the data

In [ ]:
from tensorflow.keras.layers import BatchNormalization
#from keras.layers.normalization import BatchNormalization # batch normalisation
#You have to import Batch Normalization from tensorflow.keras.layers

In [ ]:
batch_size = 128 # in each iteration, we consider 128 training examples at once
num_epochs = 12 # we iterate twelve times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth = 32 # use 32 kernels in both convolutional layers
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 128 # there will be 128 neurons in both hidden layers

In [ ]:
num_train = 60000 # there are 60000 training examples in MNIST
num_test = 10000 # there are 10000 test examples in MNIST

height, width, depth = 28, 28, 1 # MNIST images are 28x28 and greyscale
num_classes = 10 # there are 10 classes (1 per digit)

(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

X_train = X_train.reshape(X_train.shape[0], depth, height, width)
X_test = X_test.reshape(X_test.shape[0], depth, height, width)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255 # Normalise data to [0, 1] range
X_test /= 255 # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [ ]:
inp = Input(shape=(depth, height, width)) # N.B. Keras expects channel dimension first
inp_norm = BatchNormalization(axis=1)(inp) # apply BN to the input (N.B. need to rename here)
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
outs = [] # the list of ensemble outputs
for i in range(ens_models):
  conv_1 = Convolution2D(conv_depth, kernel_size, kernel_size, padding='same', kernel_initializer='he_uniform', kernel_regularizer=l2(l2_lambda), activation='relu')(inp_norm)
  conv_1 = BatchNormalization(axis=1)(conv_1)
  conv_2 = Convolution2D(conv_depth, kernel_size, kernel_size, padding='same', kernel_initializer='he_uniform', kernel_regularizer=l2(l2_lambda), activation='relu')(conv_1)
  conv_2 = BatchNormalization(axis=1)(conv_2)
  pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
  drop_1 = Dropout(drop_prob_1)(pool_1)
  flat = Flatten()(drop_1)
  hidden = Dense(hidden_size, kernel_initializer='he_uniform', kernel_regularizer=l2(l2_lambda), activation='relu')(flat) # Hidden ReLU layer
  hidden = BatchNormalization(axis=1)(hidden)
  drop = Dropout(drop_prob_2)(hidden)
  #out = Dense(num_classes, kernel_initializer='glorot_uniform', kernel_regularizer=l2(l2_lambda), activation='softmax')(drop) # Output softmax layer
  outs.append(drop) # Output softmax layer

#print(outs)
out = Concatenate()(outs)
out = Dense(num_classes, kernel_initializer='glorot_uniform', kernel_regularizer=l2(l2_lambda), activation='softmax')(out) # Output softmax layer

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

In [ ]:
model.summary()

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 1, 28, 28)]  0           []                               
                                                                                                  
 batch_normalization_206 (Batch  (None, 1, 28, 28)   4           ['input_27[0][0]']               
 Normalization)                                                                                   
                                                                                                  
 conv2d_120 (Conv2D)            (None, 1, 10, 32)    8096        ['batch_normalization_206[0][0]']
                                                                                                  
 conv2d_122 (Conv2D)            (None, 1, 10, 32)    8096        ['batch_normalization_206[

In [ ]:
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

Epoch 1/12
422/422 [==============================] - 12s 25ms/step - loss: 0.7800 - accuracy: 0.7757 - val_loss: 0.2421 - val_accuracy: 0.9497
Epoch 2/12
422/422 [==============================] - 10s 24ms/step - loss: 0.3495 - accuracy: 0.9114 - val_loss: 0.1786 - val_accuracy: 0.9660
Epoch 3/12
422/422 [==============================] - 10s 24ms/step - loss: 0.2704 - accuracy: 0.9354 - val_loss: 0.1501 - val_accuracy: 0.9745
Epoch 4/12
422/422 [==============================] - 10s 24ms/step - loss: 0.2242 - accuracy: 0.9482 - val_loss: 0.1433 - val_accuracy: 0.9763
Epoch 5/12
422/422 [==============================] - 10s 24ms/step - loss: 0.1984 - accuracy: 0.9557 - val_loss: 0.1228 - val_accuracy: 0.9813
Epoch 6/12
422/422 [==============================] - 11s 26ms/step - loss: 0.1775 - accuracy: 0.9595 - val_loss: 0.1178 - val_accuracy: 0.9793
Epoch 7/12
422/422 [==============================] - 10s 24ms/step - loss: 0.1618 - accuracy: 0.9646 - val_loss: 0.1113 - val_accuracy:

In [ ]:
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

313/313 [==============================] - 1s 4ms/step - loss: 0.0893 - accuracy: 0.9813


[0.08932994306087494, 0.9812999963760376]